## Import cuxfilter

In [ ]:
import cuxfilter
from bokeh import palettes
from cuxfilter.layouts import *

In [ ]:
# update data_dir if you have downloaded datasets elsewhere
DATA_DIR = "./data"

## Download required datasets

In [ ]:
! curl https://data.rapids.ai/viz-data/146M_predictions_v2.arrow.gz --create-dirs -o $DATA_DIR/146M_predictions_v2.arrow.gz

In [ ]:
from cuxfilter.sampledata import datasets_check

datasets_check("mortgage", base_dir=DATA_DIR)

## Define charts

In [ ]:
cux_df = cuxfilter.DataFrame.from_arrow(
    DATA_DIR + "/146M_predictions_v2.arrow"
)

In [ ]:
cux_df.data.head()

In [ ]:
geoJSONSource = "https://raw.githubusercontent.com/rapidsai/cuxfilter/GTC-2018-mortgage-visualization/javascript/demos/GTC%20demo/src/data/zip3-ms-rhs-lessprops.json"

In [ ]:
chart0 = cuxfilter.charts.choropleth(
    x="zip",
    color_column="delinquency_12_prediction",
    color_aggregate_fn="mean",
    elevation_column="current_actual_upb",
    elevation_factor=0.00001,
    elevation_aggregate_fn="sum",
    geoJSONSource=geoJSONSource,
)
chart2 = cuxfilter.charts.bar("delinquency_12_prediction", data_points=50)
chart3 = cuxfilter.charts.range_slider("borrower_credit_score", data_points=50)

### Add mappings for chart1

In [ ]:
mapper1 = {}

for val in cux_df.data.dti.unique().to_pandas().tolist():
    mapper1[int(val)] = "l_" + str(val)

chart1 = cuxfilter.charts.panel_widgets.multi_select("dti", label_map=mapper1)
# also available: panel_widgets.multi_select

## Create a dashboard object

In [ ]:
d = cux_df.dashboard(
    [chart0, chart2],
    sidebar=[chart1],
    layout=cuxfilter.layouts.feature_and_base,
    theme=cuxfilter.themes.rapids_dark,
    title="Mortgage Dashboard",
)
d.add_charts(sidebar=[chart3])

## Starting the dashboard

1. d.show('current_notebook_url:current_notebook_port') remote dashboard

2. d.app() inline within the notebook cell

Incase you need to stop the server:

- d.stop()

In [ ]:
# d.show() # notebook_url="http://localhost:8888"
# d.app() # run the dashboard within the notebook cell

![mortgage](../docs/_images/mortgage.png)

## Export the queried data into a dataframe

In [ ]:
queried_df = d.export()